In [47]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics import Accuracy
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [38]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()
        
    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        # Assign last data column to label
        label = self.data[idx, -1]
        return features, label

In [39]:
# Create an instance of the WaterDataset
dataset_train = WaterDataset('../data/water_train.csv')
dataset_test = WaterDataset('../data/water_test.csv')
# Create a DataLoader based on dataset_train
dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
)

# Create a DataLoader based on dataset_test
dataloader_test = DataLoader(
    dataset_test,
    batch_size=2,
    shuffle=True,
)

# Get a batch of features and labels
features, labels = next(iter(dataloader_train))
print(features, labels)

tensor([[0.2909, 0.4982, 0.3893, 0.4879, 0.7069, 0.2795, 0.5103, 0.4630, 0.4095],
        [0.3146, 0.3784, 0.4027, 0.5621, 0.6357, 0.2214, 0.4581, 0.6129, 0.5300]],
       dtype=torch.float64) tensor([0., 0.], dtype=torch.float64)


In [40]:
# Defining networl architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(9,16)
        self.fc2 = nn.Linear(16,8)
        self.fc3 = nn.Linear(8,1)
        
    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

In [41]:
criterion = nn.CrossEntropyLoss()

In [42]:
def train_model(optimizer, net, num_epochs):
    for epoch in range(num_epochs):
        for features, labels in dataloader_train:
            features = features.float()
            labels = labels.view(-1,1).float()
            optimizer.zero_grad()
            loss = criterion(net(features),labels)
            loss.backward()
            optimizer.step()

    return loss

In [43]:
net = Net().float()

# Define the Adam optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

tensor(-0., grad_fn=<DivBackward1>)

In [46]:
# Set up binary accuracy metric
acc = Accuracy(task='binary')

net.eval().float()
with torch.no_grad():
    for features, labels in dataloader_test:
        # Get predicted probabilities for test data batch
        outputs = net(features.float())
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1).float())

# Compute total test accuracy
test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.5904572606086731


# elu

In [49]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(9, 16)
        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.bn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8, 1)
        
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.elu(x)

        # Pass x through the second set of layers
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.elu(x)

        x = nn.functional.sigmoid(self.fc3(x))
        return x